In [1]:
import sqlite3
import pandas as pd
import math
import jieba
from pyecharts.charts import WordCloud
from pyecharts import options as opts

In [10]:
# 读取数据，链接数据库，定义常量
movie_data = pd.read_excel(r'D:\Programing\python_projects\data_analysis_basics\data_set\douban_movie_data.xlsx')
FILTER_WORDS = ['知道', '影评', '电影', '影片', '这么', '那么', '怎么', '如果', '是', '的', '地', '得', '这个', '这种', '一个',
                '时候', '什么', '一部', '这部', '没有', '还有', '片子', '觉得', 'is', 'of', 'to', 'be', '...', '\n']
file=r'D:\Programing\python_projects\data_analysis_basics\data_set\douban_comment_data.db'
conn=sqlite3.connect(file)
comment_data=pd.read_sql_query('select * from comment', conn)
comment_data.head()

,ID,TIME,MOVIEID,RATING,CONTENT,CREATOR,ADD_TIME
0,4264,2015-06-10 02:45:06,10440138,1,弱到爆…剧情、氛围烘托、表演、服化道、特效，统统弱到爆！女主和哥哥转折超级无力，给女主默默烫...,身似,2018-01-05 15:05:06
1,4265,2015-06-10 10:39:36,10440138,2,傻逼电影，看在怀旧的份儿上两星半不能更多了……一蟹不如一蟹呀,Ent,2018-01-05 15:05:06
2,4266,2015-06-10 19:33:11,10440138,2,这片存在的意义难道就是告诉我们22年过去了，人类连在商业片方面都处于停滞不前，甚至倒退的状况...,有意识的贱民,2018-01-05 15:05:06
3,4267,2015-06-10 13:22:27,10440138,1,史诗级暴虐烂片。,亿万露电,2018-01-05 15:05:06
4,4268,2015-06-16 14:01:42,10440138,1,什么jb玩意儿。。。。。。如此反智。今时今日。,Marni,2018-01-05 15:05:06


In [3]:
# 获取包含一定数据评论的电影评论的计数
def get_movie_id_list(minimum_comment):
    movie_list=comment_data['MOVIEID'].value_counts()
    movie_list=movie_list[movie_list.values>minimum_comment]
    return movie_list

# 统计所有电影的评论数
all_movie_list_count=get_movie_id_list(0)
all_movie_list_count

1292222     1780
1292052     1040
2268359     1000
1292226     1000
1292344     1000
            ... 
10757823       1
1295808        1
1304901        1
1298340        1
1293746        1
Name: MOVIEID, Length: 1762, dtype: int64

In [4]:
# 取得某电影所有评论
def get_all_comment(movie_id):
    comment_list=comment_data[comment_data['MOVIEID']==movie_id]['CONTENT']
    comment_str_all=''
    for comment in comment_list:
        comment_str_all+=comment+'\n'
    return comment_str_all

# 显示该电影评论所有内容
movie_comment=get_all_comment('1292222')
print(movie_comment)

不得不说罗伯特德尼罗演技出众，但有一个小bug，当过兵的男主角，居然右手拿枪，左眼瞄准；再就是，，，美女哪是那么容易泡滴撒，搭了两句话就可以约会了，好像人家一点也不忙一样，也许那个时代不一样吧，还有端了淫窝居然没有黑帮追杀他~，其他我倒真没看出经典，那个时代美国人精神风貌倒是展现了
无聊到死，就是讲一个失了恋的出租车司机抡枪报复“黑”社会，想谋杀参议员不成就去杀嫖客和扯皮条的，这故事跟本国的杨佳比差远了，不懂为啥评价这么高。
实在是受不鸟本片的节奏，尤其是BGM，弄得装神弄鬼结果剧情根本就不给力。和黑巷少女是同年上映，朱迪给我的感觉却大不一样，在本片里还是比较女性化的。
我喜欢搏击，肖审克，死亡诗社。
而这部影片我看的昏昏欲睡，一个精神分裂病入膏肓的人喃喃自语，丝毫不吸引人
大学语音课上看的。不喜欢。当时我就在想这男主角又讨厌又变态。看得心理直发闷。
除了他的孤独。 没看出什么。。。
PS:2017年的棒子电影《出租车司机》，进入韩国历史票房版前10，豆瓣评分9.0+。但是由于讲诉的是80年代的韩国学生运动。so，很快，豆瓣上的《出租车司机》页面就404 not found。可悲的国


In [5]:
# 根据电影评论进行分词，按关键词多少进行排列
def get_comment_keywords_counts(str_list, count):
    seg_list=list(jieba.cut(str_list))
    keywords_counts=pd.Series(seg_list)
    keywords_counts=keywords_counts[keywords_counts.str.len()>1]
    keywords_counts=keywords_counts[~keywords_counts.str.contains('|'.join(FILTER_WORDS))]
    keywords_counts=keywords_counts.value_counts()[:count]
    return keywords_counts

# 打印该电影评论排名前50的关键词
movie_comment_keywords_counts=get_comment_keywords_counts(movie_comment, 50)
print(movie_comment_keywords_counts)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Articuly\AppData\Local\Temp\jieba.cache
Loading model cost 0.898 seconds.
Prefix dict has been built succesfully.


喜欢    144
社会    142
司机    140
自己    128
孤独    116
经典    113
刺杀    103
朱迪     96
故事     95
美国     86
那个     84
无聊     83
最后     80
感觉     74
生活     68
配乐     68
纽约     68
里根     65
总统     63
雏妓     61
塞斯     58
我们     58
英雄     57
拯救     56
一样     56
年代     53
可能     52
时代     52
剧情     52
镜头     51
年轻     50
me     49
压抑     47
主角     46
无法     46
不过     45
看出     45
实在     44
理解     44
因为     44
好看     43
成为     43
完全     42
男人     42
莫西     41
这样     41
寂寞     40
看到     40
他们     39
其实     39
dtype: int64


In [7]:
# 生成词云图
def generate_word_cloud(keywords_list, path_name=None):
    wordcloud=WordCloud()
    wordcloud.add('词云图', tuple(zip(keywords_list.index, keywords_list)), word_size_range=[20, 80])
    wordcloud.set_global_opts(title_opts=opts.TitleOpts(title='电影评论'))
    wordcloud.render(path=path_name)
    print(f'生成词云图：{path_name}')

In [12]:
# 根据电影ID获得电影名称和评分
def get_movie_name_and_score(movie_id):
    movie_link='https://movie.douban.com/subject/{0}/'.format(movie_id)
    search_result=movie_data[movie_data['链接']==movie_link].iloc[0]
    movie_name=search_result['电影名']
    movie_score=search_result['评分']
    print(movie_name, movie_score)
    return movie_name, movie_score

In [14]:
# 生成该电影的词云图
movie_name, movie_score=get_movie_name_and_score('1292222')
path_name=r'D:\Programing\python_projects\data_analysis_basics\wordcloud\{0}_{1}.html'.format(movie_name, movie_score)
generate_word_cloud(movie_comment_keywords_counts, path_name)

出租车司机 8.4
生成词云图：D:\Programing\python_projects\data_analysis_basics\wordcloud\出租车司机_8.4.html
